In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

## IMPORT MODEL

In [ ]:
source_path = "./model/client_model/best.pt"
model = YOLO(source_path)

## IMPORT VIDEO

In [ ]:
video_source_path = './dataset/Video/Video_Edit_3_30_detik.mp4'
cap = cv2.VideoCapture(video_source_path)

## IMPORT PHOTO

In [ ]:
photo_source_path = './dataset/Foto/K2_AB3494ZF_jpg.rf.85eec8252e637af4b08bd271b584c014.jpg'
image = cv2.imread(photo_source_path)
img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb)
plt.axis('off') 
plt.show()

## CLASS MAPPING

In [ ]:
class_mapping = {0 : "Mobil", 1 : "Motor", 2 : "Plat Nomor"}

In [ ]:
car_box_color = (255, 0, 0)   # Warna merah
motor_box_color = (0, 255, 0)  # Warna hijau
plate_box_color = (0, 0, 255)  # Warna biru

font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.8
font_thickness = 2

## PHOTO DETECTION

In [ ]:
detections = model(image)[0]

for detection in detections.boxes.data.tolist():
    x1, y1, x2, y2, score, class_id = detection
    # Mengukur tinggi teks
    text_size = cv2.getTextSize(class_mapping[class_id], font, font_scale, font_thickness)[0]
    text_height = text_size[1]
    if class_id == 0:
        cv2.rectangle(img_rgb, (int(x1), int(y1)), (int(x2), int(y2)), car_box_color, thickness=2)
        if int(y1) >= text_height + 5:
            cv2.putText(img_rgb, class_mapping[class_id], (int(x1), int(y1) - 5), font, font_scale, car_box_color, font_thickness)
        else:
            cv2.putText(img_rgb, class_mapping[class_id], (int(x1), int(y2) + text_height + 5), font, font_scale, car_box_color, font_thickness)
    elif class_id == 1:
        cv2.rectangle(img_rgb, (int(x1), int(y1)), (int(x2), int(y2)), motor_box_color, thickness=2)
        if int(y1) >= text_height + 5:
            cv2.putText(img_rgb, class_mapping[class_id], (int(x1), int(y1) - 5), font, font_scale, motor_box_color, font_thickness)
        else:
            cv2.putText(img_rgb, class_mapping[class_id], (int(x1), int(y2) + text_height + 5), font, font_scale, motor_box_color, font_thickness)
    elif class_id == 2:
        cv2.rectangle(img_rgb, (int(x1), int(y1)), (int(x2), int(y2)), plate_box_color, thickness=2)
        if int(y1) >= text_height + 5:
            cv2.putText(img_rgb, class_mapping[class_id], (int(x1), int(y1) - 5), font, font_scale, plate_box_color, font_thickness)
        else:
            cv2.putText(img_rgb, class_mapping[class_id], (int(x1), int(y2) + text_height + 5), font, font_scale, plate_box_color, font_thickness)

In [ ]:
plt.imshow(img_rgb)
plt.axis('off') 
plt.show()

## VIDEO DETECTION

### SAVE VIDEO

In [ ]:
# Define parameters for the output video
output_video_path = './predict/predict_video_2.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 video
output_fps = 30.0  # Frames per second

# Check if video capture is opened successfully
if not cap.isOpened():
    print("Error reading video file")
    exit()  # Exit the program if there's an error

# Get video frame width and height (assuming cap is already defined)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create the video writer object
out = cv2.VideoWriter(output_video_path, fourcc, output_fps, (frame_width, frame_height))

### PREDICT VIDEO

In [ ]:
ret = True

while ret:
    ret, frame = cap.read()
    if not ret:
        break  # Keluar dari loop jika tidak ada frame yang terbaca
    
    detections = model(frame)[0]
    for detection in detections.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = detection
        # Mengukur tinggi teks
        text_size = cv2.getTextSize(class_mapping[class_id], font, font_scale, font_thickness)[0]
        text_height = text_size[1]
        if class_id == 0:
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), car_box_color, thickness=2)
            if int(y1) >= text_height + 5:
                cv2.putText(frame, class_mapping[class_id], (int(x1), int(y1) - 5), font, font_scale, car_box_color, font_thickness)
            else:
                cv2.putText(frame, class_mapping[class_id], (int(x1), int(y2) + text_height + 5), font, font_scale, car_box_color, font_thickness)
        elif class_id == 1:
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), motor_box_color, thickness=2)
            if int(y1) >= text_height + 5:
                cv2.putText(frame, class_mapping[class_id], (int(x1), int(y1) - 5), font, font_scale, motor_box_color, font_thickness)
            else:
                cv2.putText(frame, class_mapping[class_id], (int(x1), int(y2) + text_height + 5), font, font_scale, motor_box_color, font_thickness)
        elif class_id == 2:
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), plate_box_color, thickness=2)
            if int(y1) >= text_height + 5:
                cv2.putText(frame, class_mapping[class_id], (int(x1), int(y1) - 5), font, font_scale, plate_box_color, font_thickness)
            else:
                cv2.putText(frame, class_mapping[class_id], (int(x1), int(y2) + text_height + 5), font, font_scale, plate_box_color, font_thickness)

    out.write(frame) 

    ## MENAMPILKAN LIVE
    # Mendapatkan dimensi asli frame
    # original_height, original_width = frame.shape[:2]

    # # Hitung faktor skala untuk menyesuaikan frame agar muat dalam satu layar laptop
    # screen_height = 720  # Misalnya, resolusi layar laptop adalah 720p
    # screen_width = 1280  # Misalnya, resolusi layar laptop adalah 1280p
    # scale_factor = min(screen_height / original_height, screen_width / original_width)

    # # Resize frame sesuai dengan faktor skala
    # resized_frame = cv2.resize(frame, None, fx=scale_factor, fy=scale_factor)

    # # Menampilkan frame yang telah diresize
    # cv2.imshow('Resized Frame', resized_frame)

    # if cv2.waitKey(1) & 0xFF == ord('q'):  # Tunggu 1ms, jika tombol 'q' ditekan, hentikan loop
    #     break

cv2.destroyAllWindows()

cap.release() 
out.release() 
    
# Closes all the frames 
cv2.destroyAllWindows() 
   
print("The video was successfully saved") 

In [25]:
!pip freeze > requirements.txt